# HBase

## 参考文献

* *https://docs.cloudera.com/runtime/7.2.10/managing-hbase/topics/hbase-perform-scans-using-hbase-shell.html*

## 实现

每个HFile都有一个块索引，通过一个磁盘查找就可以实现查询。首先，在内存的块索引进行二分查找，确定可能包含给定健的块，然后读取磁盘块找到实际要找的健。

每次更新数据时，都会将数据记录在提交日志（commit log）中，在HBase中叫做预写日志（write-ahead log，WAL），然后才将这些数据写入内存中memstore中，一旦内存保存的写入数据累计超过了一个给定的最大值，系统会将这些数据移除内存作为HFie文件刷新到磁盘。数据移除内存后，系统会丢弃对应的提交日志，只保留为持久化到磁盘的提交日志。

因为存储文件是不可被改变的，所以无法通过移除某个健/值对简单地的删除，可行的解决办法是，做个删除标记（delete marker，又称作墓碑标记），表明给定行已经删除的事实。

返回数据是包含两部分数据合并的结果，一部分是memstore中还没有写入磁盘的数据，一部分是磁盘上存储的文件。检索时不会用到WAL，只有服务器内存中数据在服务器崩溃前没有写入到磁盘，而后进行恢复数据时才会用到WAL。

HBase架构

![HBase架构](https://bj.bcebos.com/ipic/HBase架构.jpg)

HBase有3个主要的组件，客户端库、主服务器和多个Region服务器。

master服务器负责跨region服务器的全局region的负载均衡，将繁忙的服务中的region移动负载较轻的服务器中。主服务器不是实际数据存储或者检索路径的组成部分，它仅提供了负载均衡和集群管理。此外，主服务器还提供了元数据的管理操作，例如：建表和创建列簇（Column Family）。

### 小结

HBase是一个分布式、强一致性的存储系统，具有近似最优的写性能和出色读性能。

## HBase数据模型

![](https://pic3.zhimg.com/80/v2-22e7f861a6a079a558a1fb173dbd8b9e_1440w.jpg)

* `rowkey`是HBase行记录的唯一值
* HBase必须`Column Family`，后有`Column`，`Column`必须归属到某个`Column Family`

![](https://pic4.zhimg.com/80/v2-5c2c05489dc0c4af823d26641dc0e477_1440w.jpg)

HBase本质上是一个Key-Value数据库

* key为RowKey+ColumnFamily+ColumnQualifier+Timestamp+KeyType
* 删除数据，`KeyType`设置"DELETE"

## HBase Shell

### HBase Shell基础操作

#### 查看所有表

```shell
list
```

#### 查看表的Schema

```shell
describe 'table_name'
```

#### 创建表

```shell
create 'table_nanme', 'column_family_name'
```

#### 数据写入

```shell
put 'table_name', 'row_key', 'column', 'value'
```

#### 数据读取

```shell
get 'table_name', 'rowkey'
```

#### 数据扫描

```shell
scan 'table_name'

# limit 1
scan 'table_name', {LIMIT=>1}

# scan rowkey prefix
scan 'table_name',  {ROWPREFIXFILTER => 'phone_md5'}
```

#### 禁用表

```shell
disable 'table_name'
```

#### 删除表

```shell
drop 'table_name'
```

> 删除表之前，必须禁用表

### HBase过滤器

#### LIMIT

```shell
scan 'table_name', {LIMIT=>1}
```

#### ROWPREFIXFILTER

```shell
scan 'table_name', {ROWPREFIXFILTER=>'prefix_name'}
```

#### TIMERANGE

```shell
scan 'table_name', {TIMERANGE=>[1636473600000, 1636493600000], LIMIT=>1}
```

> 按时间范围查找，时间戳的单位为毫秒

#### ValueFilter

```shell
 scan 'table_name', {COLUMNS=>'info:type', FILTER=>"ValueFilter(=, 'substring:6')", LIMIT=>1}
```

> 根据字段值查找